In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
file_dir = 'C://Users/mrale/Lydia/AnalysisProjects/Movies-ETL/'

In [3]:
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [4]:
#check how many records were pulled in
len(wiki_movies_raw)

7311

In [5]:
#Also, we should always take a look at a few individual records just to make sure that the data didn't come in horribly 
#garbled. With a DataFrame, we'd do this with the head() and tail()methods, but with a list of dicts, we need to inspect 
#the records directly.

# First 5 records
#wiki_movies_raw[:5]

# Last 5 records
#wiki_movies_raw[-5:]

# Some records in the middle
wiki_movies_raw[3600:3605]

[{'url': 'https://en.wikipedia.org/wiki/Benji:_Off_the_Leash!',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0315273/',
  'title': 'Benji: Off the Leash!',
  'Directed by': 'Joe Camp',
  'Written by': 'Joe Camp',
  'Starring': ['Benji', 'Nick Whitaker', 'Shaggy', 'Gypsy the Cockatoo'],
  'Music by': 'Antonio di Lorenzo',
  'Productioncompany ': 'Mulberry Square Productions',
  'Distributed by': 'Mulberry Square Productions',
  'Release date': ['March 26, 2004', '(', '2004-03-26', ')'],
  'Running time': '97 min',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$3,817,362'},
 {'url': 'https://en.wikipedia.org/wiki/The_Best_Thief_in_the_World',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0389796/',
  'title': 'The Best Thief in the World',
  'Directed by': 'Jacob Kornbluth',
  'Produced by': ['Tim Perrell', 'Nicola Usborne'],
  'Written by': 'Jacob Kornbluth',
  'Starring': ['Marc Rozendaal',
   'Michael Silverman',
   'David Warsh

# 8.2.2: Extract the Kaggle Data

In [6]:
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [9]:
#kaggle_metadata.head()
#kaggle_metadata.tail()
kaggle_metadata.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
7806,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,9791,tt0204137,en,Animal Factory,"A young man goes to prison and a tough, older ...",...,2000-08-01,0.0,94.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Animal Factory,False,6.6,46.0
20269,False,"{'id': 108804, 'name': 'Outpost Collection', '...",3000000,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",http://www.outpostthemovies.com/,100183,tt1418712,en,Outpost: Black Sun,A pair of investigators team up with a Special...,...,2012-01-01,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,War in Hell,Outpost: Black Sun,False,4.5,32.0
6644,False,"{'id': 168880, 'name': 'Beethoven Collection',...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",NaN,11806,tt0103786,en,Beethoven,The Newton family live in their comfortable ho...,...,1992-04-03,147214049.0,87.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The head of the family is the one with the tail.,Beethoven,False,5.6,425.0
42717,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,328327,tt5782140,en,Nick Di Paolo: Another Senseless Killing,"If you've never seen Nick Di Paolo live, this ...",...,2015-02-17,0.0,56.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Nick Di Paolo: Another Senseless Killing,False,0.0,0.0
42147,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,35992,tt0060726,en,"Munster, Go Home!",Herman discovers he's the new lord of Munster ...,...,1966-06-15,0.0,96.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,America's Funniest Family in their First Full-...,"Munster, Go Home!",False,5.7,10.0


# Clean Individual Datasets

# 8.3.1 - Data Cleaning Strategies

In [12]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [13]:
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Use wiki_movies_df.columns.tolist() and run the cell to see all of the column names that were imported. Your output should
#appear as follows:
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

# Use list Comprehensions to Filter Data

In [16]:
#modify our JSON data by restricting it to only those entries that have a director and an IMDb link. We do this with a list
#comprehension.

wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
len(wiki_movies)

7076

# 8.3.4 Revisit Functions

# 8.3.5 Create a Function to Clean the Data, Part 1

In [17]:
#write a simple function to make a copy of the movie and return it.

#call the function clean_movies, and have it take movie as a parameter.
#When we pass movie as a parameter to the dict() constructor, it reserves a new space in memory and copies all of the info 
#in movie to that new space.
#def clean_movies(movie):
    #movie_copy = dict(movie)
    
#However, we have another trick that's even better. 
#Inside the function, we can create a new local variable called movie and assit it the new copy of the parameter movie.
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    return movie

#This way, inside of the clean_movie() function, movie will refer to the local copy. Any changes we make inside 
#clean_movie() will now only affect the copy, so if we make a mistake, we still have the original, untouched movie to 
#reference.

In [18]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
7060,https://en.wikipedia.org/wiki/The_Insult_(film),2018.0,https://www.imdb.com/title/tt7048622/,The Insult,Ziad Doueiri,"[Rachid Bouchareb, Jean Bréhat, Julie Gayet, A...",NaN,NaN,NaN,"[Adel Karam, Kamel El Basha]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7293,https://en.wikipedia.org/wiki/Capernaum_(film),2018.0,https://www.imdb.com/title/tt8267604/,Capernaum,Nadine Labaki,"[Michel Merkt, Khaled Mouzanar]","[Nadine Labaki, Jihad Hojaily, Michelle Keserw...","[Georges Khabbaz, Nadine Labaki, Michelle Kese...",NaN,"[Zain Al Rafeea, Yordanos Shiferaw, Boluwatife...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

7060    https://en.wikipedia.org/wiki/The_Insult_(film)
7293     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [20]:
sorted(wiki_movies_df.columns.tolist())

['Actor control',
 'Adaptation by',
 'Alias',
 'Alma mater',
 'Also known as',
 'Animation by',
 'Arabic',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Characters',
 'Children',
 'Chinese',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Country of origin',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developed by',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Divisions',
 'Duration',
 'Edited by',
 'Editor(s)',
 'Ending theme',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'French',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Ha

# Handle the Alternative Titles

In [21]:
#step1: make an empty dict to hold all of the alternative titles.
# def clean_movies(movie):
#     movie = dict(movie) #create a non-desructive copy
#     alt_titles = {}
#     return movie

#step2: loop through a list of all alternative title keys.
# def clean_movie(movie):
#     movie = dict(movie) #create a non-destructive copy
#     alt_titles = {}
#     for key in ['Also known as','Arabic','Cantonese','Chinese','French',
#                 'Hangul','Hebrew','Hepburn','Japanese','Literally',
#                 'Mandarin','McCune–Reischauer','Original title','Polish',
#                 'Revised Romanization','Romanized','Russian',
#                 'Simplified','Traditional','Yiddish']:

#     return movie

#step2a: check if the current key exists in the movie object
# def clean_movie(movie):
#     movie = dict(movie) #create a non-destructive copy
#     alt_titles = {}
#     for key in ['Also known as','Arabic','Cantonese','Chinese','French',
#                 'Hangul','Hebrew','Hepburn','Japanese','Literally',
#                 'Mandarin','McCune–Reischauer','Original title','Polish',
#                 'Revised Romanization','Romanized','Russian',
#                 'Simplified','Traditional','Yiddish']:
#         if key in movie:

#     return movie

#step2b: if so, remove the key-value pair and add to the laternative titles dictionary.
# def clean_movie(movie):
#     movie = dict(movie) #create a non-destructive copy
#     alt_titles = {}
#     for key in ['Also known as','Arabic','Cantonese','Chinese','French',
#                 'Hangul','Hebrew','Hepburn','Japanese','Literally',
#                 'Mandarin','McCune–Reischauer','Original title','Polish',
#                 'Revised Romanization','Romanized','Russian',
#                 'Simplified','Traditional','Yiddish']:
#         if key in movie:
#             alt_titles[key] = movie[key]
#             movie.pop(key)


#     return movie

#step3: After looping through every key, add the alternative titles dict to the movie object.
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    return movie

In [22]:
#make a list of cleaned movies with a list comprehension:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [23]:
#Set wiki_movies_df to be the DataFrame created from clean_movies, and print out a list of the columns.
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']